In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import differential_entropy,kurtosis
import pandas as pd
from scipy.optimize import minimize
import statsmodels.api as sm

In [23]:
def Majew(gamma, alpha, kappa1, beta, Lambda, sigmav, length = 1000, nb = 1 ):
    #nb : nb de simulations
    # length : taille de la serie temporelle
    Epsilon=np.random.normal(0, sigmav**2, (length,nb))
    v=np.zeros((length,nb))
    m=np.zeros((length,nb))
    
    # We initialize the different functions we will need
    
    p=np.zeros((length,nb))
    r=np.zeros((length,nb))
    # We initialize the price

    for t in range(1,length-1):
        
        v[t] = (1-Lambda)*v[t-1] + Lambda*p[t]
        
        m[t] = (1-alpha)*m[t-1] + alpha*(p[t]-p[t-1])
        
        p[t+1] = p[t] + kappa1*(v[t]-p[t]) + beta*np.tanh(gamma*m[t]) + Epsilon[t+1]
        
        r[t+1] = p[t+1] - p[t]
    
    return p,r

In [36]:
def compute_statistics(r, cov=False):
    means = np.mean(r, axis=0)
    std_dev = np.std(r, axis=0)
    kurtosiss = kurtosis(r, axis=0)
    
    abs_r = np.abs(r)
    q1 = np.percentile(abs_r, 10, axis=0)
    q2 = np.percentile(abs_r, 95, axis=0)
    rat_quantile = q1/q2
    if r.shape[1] == 1 :
        aut_r = sm.tsa.acf(r, nlags= 1)[1]
        aut_rc = sm.tsa.acf(r**2, nlags= 1)[1]
    else :
        aut_r = np.array([sm.tsa.acf(r[:, i], nlags=7)[1] for i in range(r.shape[1])])
        aut_rc = np.array([sm.tsa.acf((r[:, i])**2, nlags=7)[1] for i in range(r.shape[1])])
        # if (np.sum(np.isnan(aut_r)) > 0 ):
        #     print("danger1")
        # if  (np.sum(np.isnan(aut_rc)) > 0 ) : 
        #     print("danger2")
    
    if cov:
        mf = np.vstack([means, std_dev, kurtosiss, rat_quantile, aut_r, aut_rc])
        # if np.sum(np.isnan(mf)) > 0 :
        #     print("danger")
        W = np.cov(mf)
        # if np.sum(np.isnan(W)) > 0 :
        #     print("danger")
        
        return np.array([np.mean(means), np.mean(std_dev), np.mean(kurtosiss), np.mean(rat_quantile), np.mean(aut_r), np.mean(aut_rc)]), W
    
    return np.array([np.mean(means), np.mean(std_dev), np.mean(kurtosiss), np.mean(rat_quantile), np.mean(aut_r), np.mean(aut_rc)])



In [29]:
_, r_thild = Majew(36.7,1/7,0.015,0.015,0.05,0.018)
c_thilde = compute_statistics(r_thild)
c_thilde

array([-2.75135137e-02,  1.51278800e-01,  5.96704538e+01,  6.96910058e-04,
        9.35167163e-01,  8.74999342e-01])

In [39]:
def cost(ksi):
    _, r = Majew(*ksi, nb = 100)
    if(np.sum(np.isnan(r))) : 
        print("danger")
    cabm, V = compute_statistics(r, cov=True)
    dc = c_thilde - cabm
    W = np.linalg.inv(V)
    D = dc.T @ W @ dc
    print(D)
    return D

In [40]:
options = {
    'maxiter': 1000, 
    'disp': True,
    'adaptive': False
}
init = (30.7, 1/10, 0.035, 0.1, 0.1, 0.085)
res = minimize(cost, init, method='Powell', options=options)


/tmp/ipykernel_2276/1264327979.py:7: OptimizeWarning: Unknown solver options: adaptive
  res = minimize(cost, init, method='Powell', options=options)


461872.0112184254
333537.1244787909
434165.47986895504
329077.44912636693
360809.16538606345
389473.1461136896
339868.45218256785
410547.6411720444
454526.31457790476
332832.33071052434
283216.2404886661
385694.4588318715
331966.56809934
391215.8875951579
357077.7937952792
415242.59156913654
314537.31375862926
333575.83197336516
342506.20484808664
288289.17887651623
485429.3113826613
503705.8815728154


/tmp/ipykernel_2276/2605413099.py:21: RuntimeWarning: overflow encountered in multiply
  p[t+1] = p[t] + kappa1*(v[t]-p[t]) + beta*np.tanh(gamma*m[t]) + Epsilon[t+1]
/tmp/ipykernel_2276/2605413099.py:19: RuntimeWarning: overflow encountered in multiply
  m[t] = (1-alpha)*m[t-1] + alpha*(p[t]-p[t-1])


544560.1886270127
399992.2788056577
305625.9618384302
373920.246056807
474143.02272193285
365637.8012061554
370267.39117845066
299255.36822736263
376481.43083970156
346429.1851971671
437614.56350432575
407550.6512571406
402407.90094400226
353648.658427715
333109.0939316065
399226.7411353851
332861.88696106157
390318.8782936634
294887.2559348345
422249.24379119335
284873.32667369186
309379.9800675007
342026.65309903433
438690.7775473551
333673.0561376007
486845.2373660351
436087.6111742244
393706.4233338811
353662.65044523025
66644179455117.01
danger
nan
danger


/tmp/ipykernel_2276/2605413099.py:21: RuntimeWarning: overflow encountered in add
  p[t+1] = p[t] + kappa1*(v[t]-p[t]) + beta*np.tanh(gamma*m[t]) + Epsilon[t+1]
/tmp/ipykernel_2276/2605413099.py:21: RuntimeWarning: invalid value encountered in subtract
  p[t+1] = p[t] + kappa1*(v[t]-p[t]) + beta*np.tanh(gamma*m[t]) + Epsilon[t+1]
/tmp/ipykernel_2276/3277841939.py:15: RuntimeWarning: overflow encountered in square
  aut_rc = np.array([sm.tsa.acf((r[:, i])**2, nlags=7)[1] for i in range(r.shape[1])])


nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
danger
nan
         Current function value: nan
         Iterations: 2
         Function evaluations: 81


/opt/mamba/lib/python3.12/site-packages/scipy/optimize/_minimize.py:704: RuntimeWarning: NaN result encountered.
  res = _minimize_powell(fun, x0, args, callback, bounds, **options)


In [18]:
res.x

array([30.70189499,  0.10002991,  0.03499378,  0.09998828,  0.20504312,
        0.08712036])

In [36]:
from scipy.optimize import minimize

options = {
    'maxiter': 1000, 
    'disp': True,
    'adaptive': False
}
init = (30.7,1/5,0.095,0.005,0.2,0.09)
res = minimize(cost, init, method='COBYLA', options=options)




104435.09139634557
97034.76961376292
126471.21097000648
85784.00523932056
482257.8703986979
109149.215724246
1052598.002236287
1019062.2098451274
955866.464280171
100879.15442349004
9358123.89460354
119011.37561352857
4000157.9261094434
255504.48412130147
139201.08741700152
263472.05166126357
88084.58170567123
514393.4705674616
317383.8506796428
145488.97209437747
148213.7767143904
32445.540747180246
97479.41224611962
34407.15064100759
23514.500041076393
33224.16816725979
4552.921742347499
6021.378749983205
1769.8411882021348
2138.055805985725
6888.51765531289
2297.839788698438
144.3336481960585
12213.233073772899
2664.8924550893935
1246.19691029639
152.77482179330158
2467.6360526579574
234.8577306719819
591.2384785924953
529.064757878048
3159.560147716397
348.6292904833731
239.45271141029465
830.9695451456125
255.26601612957987
158.48287122437983
29.02243233417638
7.676837321080616
23.922897758546206
9.207863521316893
38.85347600698934
49.34173674749484
17.666332924899557
17.776700164

In [37]:
print((res.x))
(36.7,1/7,0.015,0.015,0.1,0.018)

[3.17052171e+01 8.56264795e-02 1.13009811e+00 6.07987820e-02
 2.10329436e-01 1.61723828e-02]


(36.7, 0.14285714285714285, 0.015, 0.015, 0.1, 0.018)

In [19]:
# Initial guess
init_tnc = (0.02,1/9,0.01,32.7,0.3,0.02)

# Define the options for TNC
options_tnc = {
    'maxiter': 10000,
    'disp': True,
    'gtol': 1e-9
}

# Perform the optimization using 'TNC' method
res_tnc = minimize(cost, init_tnc, method='TNC', options=options_tnc)



/tmp/ipykernel_23692/3810536476.py:12: OptimizeWarning: Unknown solver options: maxiter
  res_tnc = minimize(cost, init_tnc, method='TNC', options=options_tnc)


61.82928286956156
81.33661239081829
68.05364898699754
93.0452578632954
85.05008659160599
92.21211910710814
73.94583744652749


  NIT   NF   F                       GTG
    0    1  6.182928286956156E+01   3.00284900E+19
tnc: fscale = 1.27461e-11


70.63851698943051
85.9606450257949
74.61399539142373
105.10650743913585
82.41203747000277
72.35812547883425
102.69259048889137
87.79946337140937
73.40506907939287
82.97136859652142
115.05418113659842
83.53959208225865
96.52747982637818
61.57653625288467
70.9614770409953
106.91715747119878
64.69897999247674
74.65194015804325
75.12616804738578
89.32755113667433
72.49733721759318
69.93995355315367
73.50083823696531
78.11547469303048
69.5084441032646
95.66094434114312
84.03539813831279
75.55244269661154
95.0791545576757
119.05526956686474
71.88334751151628
65.72364315119725
98.26709590664507
74.78238567971327
96.26471653202233
84.22647539661521
73.18354943008258
70.63041058261554
82.21743941913941
68.48137573175404
101.32782217806532
69.07196399555852
90.94015096702577
71.82624232149274
92.36315282791443
77.26161352890806
69.11413830344813
73.61685914196487
117.25243934003323
73.2326272748184
63.23754339164841
84.8314360205361
81.74078648865967
62.47529549309394
63.69525206110471
87.141570

    0   27  6.182928286956156E+01   3.00284900E+19
tnc: Linear search failed


81.69647484820153
76.99242378476671
69.95475681895458
92.59179003789025
73.47887898314013
67.24993597396694
95.79401023387771


In [21]:
print((res_tnc.x))

[2.00000000e-02 1.11111111e-01 1.00000000e-02 3.27000000e+01
 3.00000000e-01 2.00000000e-02]


In [86]:
def Majew2(gamma, alpha, kappa1, kappa3, beta, Lambda, sigmav):
    
    Epsilon=np.random.normal(0, sigmav**2, (length,nb))
    v=np.zeros((length,nb))
    m=np.zeros((length,nb))
    
    # We initialize the different functions we will need
    
    p=np.zeros((length,nb))
    r=np.zeros((length,nb))
    
    # We initialize the price
    
    for t in range(1,length-1):
        
        v[t] = (1-Lambda)*v[t-1] + Lambda*p[t]
        
        m[t] = (1-alpha)*m[t-1] + alpha*(p[t]-p[t-1])
        
        p[t+1] = p[t] + kappa1*(v[t]-p[t]) + kappa3*(v[t]-p[t]) + beta*np.tanh(gamma*m[t]) + Epsilon[t+1]
        
        r[t+1] = p[t+1] - p[t]
    
    return p,r